## Documentation

To read more about the SQL search API, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/sql-search-api.html).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Index data

In [ ]:
import json

from pprint import pprint

es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

operations = []
clothes_documents = json.load(open("../data/astronomy.json"))

for document in clothes_documents:
    operations.append({'index': {'_index': 'my_index'}})
    operations.append(document)

response = es.bulk(operations=operations)
pprint(response.body)

In [ ]:
count = es.count(index='my_index')
print('Number of documents in index:', count.body['count'])

## SQL search API

### 1. Simple select query

In [ ]:
query = {
    "query": "SELECT title FROM my_index ORDER BY id LIMIT 5"
}

response = es.sql.query(body=query)

for row in response['rows']:
    print(row)

### 2. Change response format

In [ ]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='json')
print(response)

In [ ]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='txt')
print(response)

In [ ]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='csv')
print(response)

In [ ]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='tsv')
print(response)

### 3. Filter results

In [ ]:
query = {
    "query": "SELECT * FROM my_index",
    "filter": {
        "term": {
            "title.keyword": "Black Holes"
        }
    }   
}

response = es.sql.query(
     body=query,
)

print(response)

### 4. Pagination

In [ ]:
query = {
    "query": "SELECT * FROM my_index ORDER BY id DESC",
    "fetch_size": 5
}

response = es.sql.query(
    body=query,
    format='json',
)
response.body

In [ ]:
while 'cursor' in response.body:
    response = es.sql.query(
        format='json',
        cursor=response.body['cursor'],
    )
    print(response.body)
    print()

### 5. Trasnlate API

In [ ]:
translate_query = {
    "query": "SELECT * FROM my_index WHERE content LIKE '%universe%' ORDER BY id DESC LIMIT 20"
}

translated_query = es.sql.translate(body=translate_query)
translated_query.body